In [1]:
using BenchmarkTools

f(n) = sin.(1:n)
g(n) = collect(sin(x) for x in 1:n)

@show f(10^6) == g(10^6)
@btime f(10^6)
@btime g(10^6);

f(10 ^ 6) == g(10 ^ 6) = true
  11.784 ms (2 allocations: 7.63 MiB)
  14.138 ms (2 allocations: 7.63 MiB)


In [4]:
@btime sum(sin, 1:10^6)
@btime sum(sin(x) for x in 1:10^6)

  11.162 ms (0 allocations: 0 bytes)
  11.542 ms (0 allocations: 0 bytes)


-0.11710952409816716

In [5]:
square(x) = x^2
@btime sum(square, 1:10^6)
@btime sum(square(x) for x in 1:10^6)

  6.800 μs (0 allocations: 0 bytes)
  4.800 ns (0 allocations: 0 bytes)


333333833333500000

In [7]:
a = (square(x) for x in 1:10^6)

Base.Generator{UnitRange{Int64}, typeof(square)}(square, 1:1000000)

In [8]:
@which Base.mapfoldl_impl(identity, Base.add_sum, Base._InitialValue(), a)

mapfoldl_impl(f::F, op::OP, nt, itr) where {F, OP} in Base at reduce.jl:42

In [9]:
@code_typed sum(square(x) for x in 1:10^6)

CodeInfo(
1 ── %1  = Base.getfield(a, :iter)::UnitRange{Int64}
│    %2  = Base.getfield(%1, :start)::Int64
│    %3  = Base.getfield(%1, :stop)::Int64
│    %4  = Base.slt_int(%3, %2)::Bool
└───       goto #3 if not %4
2 ──       goto #4
3 ── %7  = Base.getfield(%1, :start)::Int64
│    %8  = Base.getfield(%1, :start)::Int64
└───       goto #4
4 ┄─ %10 = φ (#2 => true, #3 => false)::Bool
│    %11 = φ (#3 => %7)::Int64
│    %12 = φ (#3 => %8)::Int64
└───       goto #6 if not %10
5 ──       goto #15
6 ── %15 = Base.mul_int(%11, %11)::Int64
7 ┄─ %16 = φ (#6 => %15, #13 => %31)::Int64
│    %17 = φ (#6 => %12, #13 => %25)::Int64
└───       goto #14 if not true
8 ── %19 = Base.getfield(%1, :stop)::Int64
│    %20 = (%17 === %19)::Bool
└───       goto #10 if not %20
9 ──       goto #11
10 ─ %23 = Base.add_int(%17, 1)::Int64
└───       goto #11
11 ┄ %25 = φ (#10 => %23)::Int64
│    %26 = φ (#9 => true, #10 => false)::Bool
│    %27 = φ (#10 => %23)::Int64
└───       goto #13 if not %26
12 ─       g